In [1]:
missing_files = ("1_1_2_6_25", "2_1_1_4_24", "2_1_1_6_24", "2_1_1_7_25", "2_2_1_5_30", "3_1_1_6_25", "3_1_2_4_30", "3_2_1_5_30", "4_1_2_8_25", "5_1_2_3_25", "6_1_2_2_30", "6_1_2_5_30", "6_2_1_2_25",
                 "6_2_2_13_30", "6_2_2_6_30")


In [2]:
import os
from sklearn.model_selection import train_test_split
import numpy as np
import cv2

FILENAME_PATH = r""
CSV_VIDEO_INPUT_PATH = r""
CSV_IMAGE_OUTPUT_PATH = r""

train_files = []

for root, dirs, files in os.walk(FILENAME_PATH):
    for filename in files:
        if filename.endswith(".mp4"):
            if filename.startswith(missing_files):
                continue
            else:
                train_files.append(filename.split('.')[0])

train_files, test_files = train_test_split(
    train_files, test_size=0.2, random_state=42)

train = []
val = []
test = []
for filename in test_files:
    cap = cv2.VideoCapture(CSV_VIDEO_INPUT_PATH + "\\" + filename +
                           '_' + str(1) + "_m.avi")
    f = 0
    j = 2
    while cap.isOpened() and j <= 11:
        ret, image = cap.read()
        if ret != True:
            break

        if f % 24 == 0:
            image = cv2.resize(image, (150, 100),
                               interpolation=cv2.INTER_CUBIC)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = image.flatten()
            image = image.astype(np.int32)
            image = np.insert(image, 0, filename.replace(
                "_", "")).reshape(1, 45001)

            if j % 5 == 0:
                test.append(image)
            elif j % 2 == 0:
                train.append(image)
            elif j % 3 == 0:
                val.append(image)
            else:
                train.append(image)
            j += 1

        f += 1

    cap.release()

train = np.concatenate(train, axis=0)
val = np.concatenate(val, axis=0)
test = np.concatenate(test, axis=0)

np.random.seed(42)
np.random.shuffle(train)
np.random.shuffle(val)
np.random.shuffle(test)


In [3]:
os.chdir(CSV_IMAGE_OUTPUT_PATH)
np.savetxt("train.csv", train, fmt="%d", delimiter=',')
np.savetxt("val.csv", val, fmt="%d", delimiter=',')
np.savetxt("test.csv", test, fmt="%d", delimiter=',')
